## Pandas (Aggregation and Grouping)

Una pieza esencial en el análisis de datos es una eficiente sumarización, tales como sum(), mean(), median(), min(), max()...

En esta sección se estudiará desde simples operaciones igual a las que se ejecutan para Numpy arrays a otras más sofisticadas basadas en el concepto de groupby()


In [163]:
import numpy as np
import pandas as pd
import seaborn as sns

#### Planets Data

Usaremos la base de datos Planets, que está dentro de seaborn.
Da información sobre los exoplanetas que han descubierto los
astrónomos

In [164]:
planets = sns.load_dataset('planets')

In [165]:
type(planets)

pandas.core.frame.DataFrame

In [166]:
planets.shape

(1035, 6)

In [167]:
planets.head()

,method,number,orbital_period,mass,distance,year
0,Radial Velocity,1,269.300,7.10,77.40,2006
1,Radial Velocity,1,874.774,2.21,56.95,2008
2,Radial Velocity,1,763.000,2.60,19.84,2011
3,Radial Velocity,1,326.030,19.40,110.62,2007
4,Radial Velocity,1,516.220,10.50,119.47,2009


### Simple Aggregation in Pandas

Parecido a lo que podemos hacer con Numpy arrays, podemos hacer con las Series y DataFrames de Pandas

In [168]:
rng = np.random.RandomState(42)

In [169]:
ser = pd.Series(rng.rand(5))

In [170]:
ser

0    0.374540
1    0.950714
2    0.731994
3    0.598658
4    0.156019
dtype: float64

In [171]:
# suma
ser.sum()

2.811925491708157

In [172]:
# media
ser.mean()

0.5623850983416314

In [173]:
df = pd.DataFrame({'A': rng.rand(5),
                   'B': rng.rand(5)})

In [174]:
df

,A,B
0,0.155995,0.020584
1,0.058084,0.969910
2,0.866176,0.832443
3,0.601115,0.212339
4,0.708073,0.181825


In [175]:
df.mean() # agrega por columnas

A    0.477888
B    0.443420
dtype: float64

In [176]:
df.mean(axis='columns') # agrega por filas

0    0.088290
1    0.513997
2    0.849309
3    0.406727
4    0.444949
dtype: float64

In [177]:
(df.values[0,0] + df.values[0,1]) / 2 

0.08828950731600255

#### DataFrame describe() method

Pandas tiene un método llamado describe() que calcula una serie de estadísticos para cada una de las columnas y devuelve un dataframe con el resultado

In [178]:
planets.describe()

,number,orbital_period,mass,distance,year
count,1035.000000,992.000000,513.000000,808.000000,1035.000000
mean,1.785507,2002.917596,2.638161,264.069282,2009.070531
std,1.240976,26014.728304,3.818617,733.116493,3.972567
min,1.000000,0.090706,0.003600,1.350000,1989.000000
25%,1.000000,5.442540,0.229000,32.560000,2007.000000
50%,1.000000,39.979500,1.260000,55.250000,2010.000000
75%,2.000000,526.005000,3.040000,178.500000,2012.000000
max,7.000000,730000.000000,25.000000,8500.000000,2014.000000


In [179]:
# echamos un vistazo por si existen valores nulos
planets.isnull().any(axis=0)

method            False
number            False
orbital_period     True
mass               True
distance           True
year              False
dtype: bool

In [180]:
# eliminamos los valores nulos antes de calcular los estadísticos
planets.dropna().describe()

,number,orbital_period,mass,distance,year
count,498.00000,498.000000,498.000000,498.000000,498.000000
mean,1.73494,835.778671,2.509320,52.068213,2007.377510
std,1.17572,1469.128259,3.636274,46.596041,4.167284
min,1.00000,1.328300,0.003600,1.350000,1989.000000
25%,1.00000,38.272250,0.212500,24.497500,2005.000000
50%,1.00000,357.000000,1.245000,39.940000,2009.000000
75%,2.00000,999.600000,2.867500,59.332500,2011.000000
max,6.00000,17337.500000,25.000000,354.000000,2014.000000


#### Aggregation Methods

Existen una serie de métodos básicos, tanto para Series como para DataFrames:

* count() : número total de items
* first(), last(): primer y último item
* mean(),median(): media y mediana
* min(), max(): valor mínimo y máxido
* std(), var() : desviación standar y varianza
* mad() : desviación absoluta de la media
* prod() : producto de todos los items
* sum() : suma de todos los items

Para ir más allá en los datos, estas simples asociaciones no son suficientes y necesitamos de un siguiente nivel, que se consigue con el método groupby()


### Group By: Split, Apply, Combine

A menudo vamos a necesitar realizar agregaciones en función de una columna concreta o indice. Esto está implementado a través de la función groupby().

Podemos pensar en esta funcionalidad como un proceso de:

* Split: dividir y agrupar un dataframe en función de una clave
* Apply: aplicar algunas funciones, generalmente agregación, transformación y/o filtrado de los grupos obtenidos a través del Split
* Combine: unir el resultado de estas operaciones en un nuevo array




In [181]:
df = pd.DataFrame({'key':['A','B','C','A','B','C'],
                  'data': range(6)}, columns=['key','data'])

In [182]:
df

,key,data
0,A,0
1,B,1
2,C,2
3,A,3
4,B,4
5,C,5


In [183]:
# hacemos una agrupación por la columna 'key'
df.groupby('key')

##### Split

No devuelve un nuevo df, sino que devuelve un objeto DataFrameGroupBy. Es una vista especial del df original, que no realiza el cálculo hasta que se realiza la función de agregación. Es una "lazy evaluation" hasta que no se computa el cálculo.

##### Apply - Combine

Para producir el resultado, tenemos que realizar una operación de agregación

In [184]:
# vamos a sumar los valores de la columna
df.groupby('key').sum()

,data
key,
A,3
B,5
C,7


#### The GroupBy object

La utilidad del objeto es que nos permite realizar las operaciones de:
* Agregación
* Filtrado
* Transformación
* Aplicación

##### Column Indexing

El objeto soporta la funcionalidad de indexado, del mismo modo que un DataFrame, devolviendo un objeto GroupBy modificado

In [185]:
# Agrupamos por la columna 'method'
planets.groupby('method')

In [186]:
# Seleccionamos la columna 'orbital_period'
planets.groupby('method')['orbital_period']

In [187]:
# Calculamos la mediana de cada período orbital para cada uno de los
# métodos
planets.groupby('method')['orbital_period'].median()

method
Astrometry                         631.180000
Eclipse Timing Variations         4343.500000
Imaging                          27500.000000
Microlensing                      3300.000000
Orbital Brightness Modulation        0.342887
Pulsar Timing                       66.541900
Pulsation Timing Variations       1170.000000
Radial Velocity                    360.200000
Transit                              5.714932
Transit Timing Variations           57.011000
Name: orbital_period, dtype: float64

##### Iterating over groups

El objeto soporta la iteración directa por cada uno de los grupos, retornando cada grupo una Serie o DataFrame

In [188]:
for (method, group) in planets.groupby('method'):
    print("{0:30s} shape={1}".format(method, group.shape))

Astrometry                     shape=(2, 6)
Eclipse Timing Variations      shape=(9, 6)
Imaging                        shape=(38, 6)
Microlensing                   shape=(23, 6)
Orbital Brightness Modulation  shape=(3, 6)
Pulsar Timing                  shape=(5, 6)
Pulsation Timing Variations    shape=(1, 6)
Radial Velocity                shape=(553, 6)
Transit                        shape=(397, 6)
Transit Timing Variations      shape=(4, 6)


##### Dispatch methods

Podemos aplicar métodos a los grupos resultantes en función de si estos son una Serie o un DataFrame

In [189]:
planets.groupby('method')['year'].describe()

,count,mean,std,min,25%,50%,75%,max
method,,,,,,,,
Astrometry,2.0,2011.500000,2.121320,2010.0,2010.75,2011.5,2012.25,2013.0
Eclipse Timing Variations,9.0,2010.000000,1.414214,2008.0,2009.00,2010.0,2011.00,2012.0
Imaging,38.0,2009.131579,2.781901,2004.0,2008.00,2009.0,2011.00,2013.0
Microlensing,23.0,2009.782609,2.859697,2004.0,2008.00,2010.0,2012.00,2013.0
Orbital Brightness Modulation,3.0,2011.666667,1.154701,2011.0,2011.00,2011.0,2012.00,2013.0
Pulsar Timing,5.0,1998.400000,8.384510,1992.0,1992.00,1994.0,2003.00,2011.0
Pulsation Timing Variations,1.0,2007.000000,NaN,2007.0,2007.00,2007.0,2007.00,2007.0
Radial Velocity,553.0,2007.518987,4.249052,1989.0,2005.00,2009.0,2011.00,2014.0
Transit,397.0,2011.236776,2.077867,2002.0,2010.00,2012.0,2013.00,2014.0


In [190]:
planets.groupby('method')['year'].describe().unstack()

       method                       
count  Astrometry                          2.000000
       Eclipse Timing Variations           9.000000
       Imaging                            38.000000
       Microlensing                       23.000000
       Orbital Brightness Modulation       3.000000
       Pulsar Timing                       5.000000
       Pulsation Timing Variations         1.000000
       Radial Velocity                   553.000000
       Transit                           397.000000
       Transit Timing Variations           4.000000
mean   Astrometry                       2011.500000
       Eclipse Timing Variations        2010.000000
       Imaging                          2009.131579
       Microlensing                     2009.782609
       Orbital Brightness Modulation    2011.666667
       Pulsar Timing                    1998.400000
       Pulsation Timing Variations      2007.000000
       Radial Velocity                  2007.518987
       Transit             

#### Aggregate, Filtrate, Transform, Apply

La utilidad del objeto es que nos permite realizar las operaciones de:
* Agregación
* Filtrado
* Transformación
* Aplicación

In [191]:
df = pd.DataFrame({'key':['A','B','C','A','B','C'],
                  'data1': range(6),
                  'data2': rng.randint(0,10,6)}, 
                  columns=['key','data1', 'data2'])

In [192]:
df

,key,data1,data2
0,A,0,4
1,B,1,0
2,C,2,9
3,A,3,5
4,B,4,8
5,C,5,0


#### Aggregation

El método aggregate() permite una mayor flexibilidad a la hora de realizar agregaciones. Este puede tomar un string, una función o una lista de todo lo anterior y computar todas las agregaciones de una vez.

In [193]:
# calculamos el mínimo, media y máximo
df.groupby('key').aggregate(['min', np.median, max])

data1            data2           
      min median max   min median max
key                                  
A       0    1.5   3     4    4.5   5
B       1    2.5   4     0    4.0   8
C       2    3.5   5     0    4.5   9

In [194]:
df

,key,data1,data2
0,A,0,4
1,B,1,0
2,C,2,9
3,A,3,5
4,B,4,8
5,C,5,0


Agregación a través de un diccionario

In [195]:
df.groupby('key').aggregate({'data1': 'min',
                            'data2': 'max'})

,data1,data2
key,,
A,0,5
B,1,8
C,2,9


#### Filtering

Nos permite eliminar información en base a la agrupación realizada. Realizamos una agrupación de los datos y sobre agregaciones de esta y el resultado, devolvemos un nuevo dataframe o serie.

In [196]:
df

,key,data1,data2
0,A,0,4
1,B,1,0
2,C,2,9
3,A,3,5
4,B,4,8
5,C,5,0


In [197]:
# calcula la std de la columna data2 en base a los grupos
df.groupby('key')['data2'].std()

key
A    0.707107
B    5.656854
C    6.363961
Name: data2, dtype: float64

In [198]:
# recibe las filas de un grupo y calcula si la desviación 
# standar de cada grupo supera el valor de 4 y en ese 
# caso, lo devuelve como válido
def filter_func(x):
    print(x)
    return x['data2'].std() > 4
        

In [199]:
# filtra la información en base a una agrupación y devuelve
# un nuevo dataframe con las filas que quedan en base al filtrado
df.groupby('key').filter(filter_func)

  key  data1  data2
0   A      0      4
3   A      3      5
  key  data1  data2
1   B      1      0
4   B      4      8
  key  data1  data2
2   C      2      9
5   C      5      0


,key,data1,data2
1,B,1,0
2,C,2,9
4,B,4,8
5,C,5,0


In [200]:
df2 = df.groupby('key').filter(filter_func)

  key  data1  data2
0   A      0      4
3   A      3      5
  key  data1  data2
1   B      1      0
4   B      4      8
  key  data1  data2
2   C      2      9
5   C      5      0


In [201]:
type(df2)

pandas.core.frame.DataFrame

#### Transformation

Mientras la agregación devuelve una versión reducida de los datos, el proceso de transformación puede devolver una versión recombinada de todo el set de datos, pudiendo obtener una matriz con la misma shape.

Un ejemplo común de esto es centrar los datos en base a restar la media del grupo.

In [202]:
# agrupa por la clave y transforma los valores de las columnas
# centrando el valor sobre la media del grupo
transform = df.groupby('key').transform(lambda x: x - x.mean())

In [203]:
transform 

,data1,data2
0,-1.5,-0.5
1,-1.5,-4.0
2,-1.5,4.5
3,1.5,0.5
4,1.5,4.0
5,1.5,-4.5


In [204]:
type(transform)

pandas.core.frame.DataFrame

In [205]:
# seleccionamos una de las columnas
test = df.groupby('key')['data1'].transform(lambda x: x - x.mean())

In [206]:
test

0   -1.5
1   -1.5
2   -1.5
3    1.5
4    1.5
5    1.5
Name: data1, dtype: float64

In [207]:
type(test)

pandas.core.series.Series

#### The apply() method

Permite aplicar una función a un grupo de datos. La función debería tomar un DataFrame y devolver un objeto Pandas (Serie o DataFrame) o un  valor escalar

In [208]:
def norm_by_data2(x):
    x['data1'] /= x['data2'].sum()
    return x

In [209]:
df.groupby('key').apply(norm_by_data2)

,key,data1,data2
0,A,0.000000,4
1,B,0.125000,0
2,C,0.222222,9
3,A,0.333333,5
4,B,0.500000,8
5,C,0.555556,0


In [210]:
df.groupby('key')['data2'].sum()

key
A    9
B    8
C    9
Name: data2, dtype: int64

In [211]:
df

,key,data1,data2
0,A,0,4
1,B,1,0
2,C,2,9
3,A,3,5
4,B,4,8
5,C,5,0


In [212]:
df.values[1,0]

'B'

In [213]:
df.iloc[1,1]

1

In [214]:
df[df['key'] == 'B']['data2'].sum()

8

In [215]:
# valor de data1 en index1 (1) / suma de los datos de data2 para B (8)
1/8 

0.125

### Especificando la clave de agrupación

La clave de agrupación puede ser a partir de una simple columna u otras opciones más complejas

#### List, Array, Series or Index providing the grouping keys

Le podemos pasar una serie de valores con los que agrupar cada una de las filas del DataFrame, le asignamos este valor a cada una de ellas y luego usamos este array para agrupar. La longitud del array que hemos creado para agrupar tiene que coincidir con la longitud del Dataframe.

In [216]:
df

,key,data1,data2
0,A,0,4
1,B,1,0
2,C,2,9
3,A,3,5
4,B,4,8
5,C,5,0


In [217]:
# creamos una serie de agrupación basada en numeros
L = [0,1,0,1,2,0] 

In [218]:
# agrupamos los grupos creados anteriormente
df.groupby(L).sum()

,data1,data2
0,7,13
1,4,5
2,4,8


In [219]:
# creamos una serie de agrupación basada en cadenas de texto
L2 = ['Perro','Caballo','Perro','Caballo','Gato','Perro'] 

In [220]:
df.groupby(L2).sum()

,data1,data2
Caballo,4,5
Gato,4,8
Perro,7,13


In [221]:
# Creamos una serie cuya longitud no coincide con la del DataFrame
L3 = ['Perro','Caballo','Perro','Caballo','Gato','Perro', 'Gato'] 

In [222]:
df.groupby(L3).sum()

KeyError: 'Perro'

#### A dictionary or series mapping index to group

Podemos pasarle un diccionario que mapea los valores del índice a las claves de agrupación.

In [223]:
df2 = df.set_index('key')

In [224]:
df2

,data1,data2
key,,
A,0,4
B,1,0
C,2,9
A,3,5
B,4,8
C,5,0


In [225]:
mapping = {'A': 'vowel', 'B': 'consonant', 'C': 'consonant'}

In [226]:
# agrupamos en función del diccionario que aplicamos sobre el índice
df2.groupby(mapping).sum()

,data1,data2
consonant,12,17
vowel,3,9


#### Any Python function.

Similar al mapping, podemos pasar una función que actuará sobre el índice y el resultado lo mostrará en los grupos.

In [227]:
# convierte los valores de los índices a mínusculas y lo muestra
# en el resultado de la agrupación
df2.groupby(str.lower).sum()

,data1,data2
a,3,9
b,5,8
c,7,9


#### A list de keys validas.

Podemos agrupar generando un multi-indice como resultado de los grupos

In [228]:
# la agrupación la devuelve a minúsculas y mapeada con el diccionario
df2.groupby([str.lower, mapping]).sum()

,,data1,data2
a,vowel,3,9
b,consonant,5,8
c,consonant,7,9


#### Grouping example with Planets

Agrupa los descubrimientos de planetas en base al método de 
descubrimiento y la decada en que se produjo

In [229]:
# Creamos una serie con las décadas que usaremos como key
# de agrupación
decade = 10 * (planets['year'] // 10)

In [230]:
type(decade)

pandas.core.series.Series

In [231]:
# Converimos a string la serie añadiendo la 's'
decade = decade.astype(str) + 's'

In [232]:
decade.head()

0    2000s
1    2000s
2    2010s
3    2000s
4    2000s
Name: year, dtype: object

In [233]:
# damos nombre a la serie
decade.name = 'decade'

In [234]:
planets.head()

,method,number,orbital_period,mass,distance,year
0,Radial Velocity,1,269.300,7.10,77.40,2006
1,Radial Velocity,1,874.774,2.21,56.95,2008
2,Radial Velocity,1,763.000,2.60,19.84,2011
3,Radial Velocity,1,326.030,19.40,110.62,2007
4,Radial Velocity,1,516.220,10.50,119.47,2009


In [235]:
# agrupa por la columna 'method' y a su vez por una agrupación 
# nueva que hemos creado que es la década (agrupando por índices
# a partir de un array, serie, etc.)

# sumamos los valores en base a la agrupación
planets.groupby(['method', decade])['number'].sum()

method                         decade
Astrometry                     2010s       2
Eclipse Timing Variations      2000s       5
                               2010s      10
Imaging                        2000s      29
                               2010s      21
Microlensing                   2000s      12
                               2010s      15
Orbital Brightness Modulation  2010s       5
Pulsar Timing                  1990s       9
                               2000s       1
                               2010s       1
Pulsation Timing Variations    2000s       1
Radial Velocity                1980s       1
                               1990s      52
                               2000s     475
                               2010s     424
Transit                        2000s      64
                               2010s     712
Transit Timing Variations      2010s       9
Name: number, dtype: int64

In [236]:
# Convierte el multi-indice una df bidimensional (unstack)
planets.groupby(['method', decade])['number'].sum().unstack()

decade,1980s,1990s,2000s,2010s
method,,,,
Astrometry,NaN,NaN,NaN,2.0
Eclipse Timing Variations,NaN,NaN,5.0,10.0
Imaging,NaN,NaN,29.0,21.0
Microlensing,NaN,NaN,12.0,15.0
Orbital Brightness Modulation,NaN,NaN,NaN,5.0
Pulsar Timing,NaN,9.0,1.0,1.0
Pulsation Timing Variations,NaN,NaN,1.0,NaN
Radial Velocity,1.0,52.0,475.0,424.0
Transit,NaN,NaN,64.0,712.0


In [237]:
# rellena los nulos con 0 (fillna())
planets.groupby(['method', decade])['number'].sum().unstack().fillna(0)

decade,1980s,1990s,2000s,2010s
method,,,,
Astrometry,0.0,0.0,0.0,2.0
Eclipse Timing Variations,0.0,0.0,5.0,10.0
Imaging,0.0,0.0,29.0,21.0
Microlensing,0.0,0.0,12.0,15.0
Orbital Brightness Modulation,0.0,0.0,0.0,5.0
Pulsar Timing,0.0,9.0,1.0,1.0
Pulsation Timing Variations,0.0,0.0,1.0,0.0
Radial Velocity,1.0,52.0,475.0,424.0
Transit,0.0,0.0,64.0,712.0
